In [ ]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"

In [ ]:
!unzip drugsCom_raw.zip

In [ ]:
from datasets import load_dataset

In [ ]:
data_files = {
    'train': 'data/drugsComTrain_raw.tsv',
    'test': 'data/drugsComTest_raw.tsv'
}

In [ ]:
drug_dataset = load_dataset('csv',data_files=data_files,delimiter='\t')

In [ ]:
drug_dataset

In [ ]:
drug_sample = drug_dataset['train'].shuffle(seed=42).select(range(1000))

In [ ]:
drug_sample

In [ ]:
drug_sample[:3]

In [ ]:
drug_dataset.keys()

In [ ]:
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

In [ ]:
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)
drug_dataset

In [ ]:
drug_sample[:3]

In [ ]:
def lower_condition(example):
    return {'condition': example['condition'].lower()}

In [ ]:
drug_dataset.map(lower_condition)

In [ ]:
def filter_nones(x):
    return x['condition'] is not None

In [ ]:
drug_dataset = drug_dataset.filter(filter_nones)

In [ ]:
drug_dataset = drug_dataset.map(lower_condition)

In [ ]:
drug_dataset['train']['condition'][:3]

In [ ]:
drug_dataset['train'][:3]

In [ ]:
def compute_review_length(example):
    return {'review_length': len(example['review'].split())}

In [ ]:
drug_dataset = drug_dataset.map(compute_review_length)

In [ ]:
drug_dataset['train'][0]

In [ ]:
drug_dataset['train'].sort('review_length')[-1]

In [ ]:
drug_dataset = drug_dataset.filter(lambda x: x['review_length'] > 50)

In [ ]:
drug_dataset

In [ ]:
drug_dataset.num_rows

In [ ]:
import html

In [ ]:

%time drug_dataset = drug_dataset.map(lambda x: {'review': html.unescape(x['review'])})

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['review'],truncation=True)

In [ ]:
%time tokenized_dataset = drug_dataset.map(tokenize_function)

In [ ]:
%time tokenized_dataset = drug_dataset.map(tokenize_function,batched=True)

In [ ]:
slow_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased',use_fast=False)

In [ ]:
def slow_tokenize_function(examples):
    return slow_tokenizer(examples['review'],truncation=True)

In [ ]:
%time tokenized_dataset2 = drug_dataset.map(slow_tokenize_function,batched=True)

In [ ]:
%time tokenized_dataset2 = drug_dataset.map(slow_tokenize_function,batched=True,num_proc=8)

In [ ]:
def tokenize_and_split(examples):
    return tokenizer(
        examples['review'],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

In [ ]:
result = tokenize_and_split(drug_dataset['train'][0])

In [ ]:
[len(nip) for nip in result['input_ids']]

In [ ]:
tokenized_dataset = drug_dataset.map(tokenize_and_split,batched=True)

In [ ]:
tokenized_dataset = drug_dataset.map(tokenize_and_split,batched=True,remove_columns=drug_dataset['train'].column_names)

In [ ]:
len(tokenized_dataset['train']),len(drug_dataset['train'])

In [ ]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples['review'],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    sample_map = result.pop('overflow_to_sample_mapping')
    for key,values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

In [ ]:
tokenized_dataset = drug_dataset.map(tokenize_and_split,batched=True)

In [ ]:
tokenized_dataset

In [ ]:
drug_dataset.set_format('pandas')

In [ ]:
drug_dataset

In [ ]:
drug_dataset['train'][:3]

In [ ]:
train_df = drug_dataset['train'][:]

In [ ]:
train_df

In [ ]:
frequencies = (
    train_df['condition']
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={'index': 'condition','count': 'frequency'}))

In [ ]:
frequencies.head()

In [ ]:
from datasets import Dataset

In [ ]:
freq_dataset = Dataset.from_pandas(frequencies)

In [ ]:
freq_dataset[:10]

In [ ]:
drug_dataset

In [ ]:
drug_dataset.reset_format()

In [ ]:
type(drug_dataset)

In [ ]:
drug_data_clean = drug_dataset['train'].train_test_split(train_size=0.8,seed=42)

In [ ]:
drug_data_clean['validation'] = drug_data_clean.pop('test')

In [ ]:
drug_data_clean['test'] = drug_dataset['test']

In [ ]:
drug_data_clean

In [ ]:
drug_data_clean.save_to_disk('data/drug-reviews')

In [ ]:
from datasets import load_from_disk

In [ ]:
drug_data_reloaded = load_from_disk('data/drug-reviews')

In [ ]:
drug_data_reloaded

In [ ]:
for split,dataset in drug_data_clean.items():
    dataset.to_json(f'data/json/drug-reviews-{split}.jsonl')

In [ ]:
!head -n 1 data/json/drug-reviews-train.jsonl

In [ ]:
data_files = {
    "train": "data/json/drug-reviews-train.jsonl",
    "validation": "data/json/drug-reviews-validation.jsonl",
    "test": "data/json/drug-reviews-test.jsonl",
}
drug_dataset_reloaded = load_dataset("json", data_files=data_files)

In [ ]:
drug_dataset_reloaded